## Validate Azure ML SDK installation and get version number for debugging purposes

In [ ]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [ ]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

## Initialize Workspace
Initialize a workspace object from persisted configuration.

In [ ]:
# Initialize Workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print("Resource group: ", ws.resource_group)
print("Location: ", ws.location)
print("Workspace name: ", ws.name)

## Create An Experiment
**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'distributed-tensorflow'
experiment = Experiment(workspace = ws, name = experiment_name)

## Create Azure Batch AI cluster (GPU-enabled) as a compute target

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_target_name = 'myazbai'

try:
    batch_ai_compute = AmlCompute(workspace=ws, name=compute_target_name)
    print('found existing:', batch_ai_compute.name)
except ComputeTargetException:
    print('creating new.')
    batch_ai_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_NC6",
        vm_priority="dedicated",
        min_nodes = 0,
        max_nodes = 4
    )
    batch_ai_compute = AmlCompute.create(
        ws, 
        name=compute_target_name, 
        provisioning_configuration=batch_ai_config
    )
    batch_ai_compute.wait_for_completion(show_output=True)

## Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [ ]:
import os

project_folder = '../projects/tf-distr-ps'
os.makedirs(project_folder, exist_ok=True)

In [ ]:
import shutil
shutil.copy('./scripts/tf_mnist_replica.py', project_folder)

## Create a TensorFlow estimator
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-tensorflow).

In [ ]:
from azureml.train.dnn import TensorFlow

script_params={
    '--num_gpus': 1
}

estimator = TensorFlow(source_directory=project_folder,
                       compute_target=batch_ai_compute,
                       script_params=script_params,
                       entry_script='tf_mnist_replica.py',
                       node_count=2,
                       worker_count=2,
                       parameter_server_count=1,   
                       distributed_backend='ps',
                       use_gpu=True)

## Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [ ]:
run = experiment.submit(estimator)
print(run.get_details())

## Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

Alternatively, you can block until the script has completed training before running more code.

In [ ]:
run.wait_for_completion(show_output=True)